In [1]:
import pandas as pd
input_df = pd.read_csv('input.csv')

input_df.head()

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,1/1/2021,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,1/1/2021,NaN,20000,10000.0,1/1/2022,20000.0,1/1/2023,9.0,6/1/2021,9.5,6/1/2022,4.0,3/1/2021,5.0,3/1/2022
2,3,1.0,1/1/2021,12/31/2023,20000,10000.0,1/1/2022,20000.0,1/1/2023,9.0,6/1/2021,9.5,6/1/2022,4.0,3/1/2021,5.0,3/1/2022


In [4]:
from datetime import datetime, timedelta
far_future_date = '2100-01-01'

# Preprocess the input DataFrame: Convert date columns to datetime

date_columns = [col for col in input_df.columns if 'date' in col] + ['Date of Joining', 'Date of Exit']
for col in date_columns:
    input_df[col] = pd.to_datetime(input_df[col], errors='coerce', format='%m/%d/%Y')
    
# Helper function to add historical records for an employee

def add_historical_records(employee_data):
    historical_records = []
    
# Prepare data for iteration
    
    compensation_dates = [employee_data[col] for col in ['Compensation 1 date', 'Compensation 2 date'] if not pd.isna(employee_data[col])]
    review_dates = [employee_data[col] for col in ['Review 1 date', 'Review 2 date'] if not pd.isna(employee_data[col])]
    engagement_dates = [employee_data[col] for col in ['Engagement 1 date', 'Engagement 2 date'] if not pd.isna(employee_data[col])]
    
# Combine and sort unique dates
    unique_dates = sorted(set(compensation_dates + review_dates + engagement_dates))
    
    for i, effective_date in enumerate(unique_dates):
        
 # Determine the end date

        if i < len(unique_dates) - 1:
            end_date = unique_dates[i + 1] - timedelta(days=1)
        else:
            
# If this is the last date, set the end date to a far future date

            end_date = datetime.strptime(far_future_date, '%Y-%m-%d')
    
        historical_record = {
            'Employee Code': employee_data['Employee Code'],
            'Effective Date': effective_date,
            'End Date': end_date,
            # Assuming initial values are inherited unless a specific date changes them
            'Compensation': employee_data['Compensation'],
            'Review Score': employee_data['Review 1'] if not pd.isna(employee_data['Review 1']) 
            and effective_date >= employee_data['Review 1 date'] else None,
            'Engagement Score': employee_data['Engagement 1'] if not 
            pd.isna(employee_data['Engagement 1']) and effective_date >= employee_data['Engagement 1 date'] else None
        }
        
        # Update values based on the effective date
        for col in ['Compensation 1', 'Compensation 2']:
            if not pd.isna(employee_data[col]) and effective_date >= employee_data[f'{col} date']:
                historical_record['Compensation'] = employee_data[col]
                
        for col in ['Review 2']:
            if not pd.isna(employee_data[col]) and effective_date >= employee_data[f'{col} date']:
                historical_record['Review Score'] = employee_data[col]
                
        for col in ['Engagement 2']:
            if not pd.isna(employee_data[col]) and effective_date >= employee_data[f'{col} date']:
                historical_record['Engagement Score'] = employee_data[col]
        
        historical_records.append(historical_record)
    
    return historical_records

# function to each employee and collect all historical records
all_historical_records = []
for index, row in input_df.iterrows():
    all_historical_records.extend(add_historical_records(row))

# DataFrame from the historical records
historical_df = pd.DataFrame(all_historical_records)

# Converted date columns back to string format for CSV output
historical_df['Effective Date'] = historical_df['Effective Date'].dt.strftime('%Y-%m-%d')
historical_df['End Date'] = historical_df['End Date'].dt.strftime('%Y-%m-%d')

# first few rows of the transformed DataFrame
historical_df.head()


,Employee Code,Effective Date,End Date,Compensation,Review Score,Engagement Score
0,2,2021-03-01,2021-05-31,20000.0,NaN,4.0
1,2,2021-06-01,2021-12-31,20000.0,9.0,4.0
2,2,2022-01-01,2022-02-28,10000.0,9.0,4.0
3,2,2022-03-01,2022-05-31,10000.0,9.0,5.0
4,2,2022-06-01,2022-12-31,10000.0,9.5,5.0


In [3]:
# Define the path for the output CSV file
output_csv_path = 'transformed_historical_data.csv'

# Save the transformed DataFrame to a CSV file
historical_df.to_csv(output_csv_path, index=False)

# Return the path for download
output_csv_path


'transformed_historical_data.csv'